# Scraping Concerts - Lab

## Introduction

Now that you've seen how to scrape a simple website, it's time to again practice those skills on a full-fledged site!
In this lab, you'll practice your scraping skills on a music website: https://www.residentadvisor.net.
## Objectives

You will be able to:
* Create a full scraping pipeline that involves traversing over many pages of a website, dealing with errors and storing data

## View the Website

For this lab, you'll be scraping the https://www.residentadvisor.net website. Start by navigating to the events page [here](https://www.residentadvisor.net/events) in your browser.

<img src="images/ra.png">

In [ ]:
# Load the https://www.residentadvisor.net/events page in your browser.


## Open the Inspect Element Feature

Next, open the inspect element feature from your web browser in order to preview the underlying HTML associated with the page.

In [2]:
# Open the inspect element feature in your browser


## Write a Function to Scrape all of the Events on the Given Page Events Page

The function should return a Pandas DataFrame with columns for the Event_Name, Venue, Event_Date and Number_of_Attendees.

In [3]:
import re # regular expressions operations
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [11]:
html_page = requests.get("https://www.residentadvisor.net/events/us/newyork")## without location, will default to wherever you are
soup = BeautifulSoup(html_page.content, 'html.parser')
event_listings = soup.find('div', id="event-listing")
entries = event_listings.findAll('li')
print(len(entries), entries[0])


34 <li><p class="eventDate date"><a href="/events.aspx?ai=8&amp;v=day&amp;mn=4&amp;yr=2020&amp;dy=20"><span>Mon, 20 Apr 2020 /</span></a></p></li>


In [12]:
rows = []
for entry in entries:
    date = entry.find('p', class_="eventDate date") 
    event = entry.find('h1', class_="event-title")
    if event:
        details = event.text.split(' at ')
        event_name = details[0].strip()
        venue = details[1].strip()
        try:
            n_attendees = int(re.match("(\d*)", entry.find('p', class_="attending").text)[0])
        except:
            n_attendees = np.nan
        rows.append([event_name, venue, cur_date, n_attendees])
    elif date:
        cur_date = date.text
    else:
        continue
df = pd.DataFrame(rows)
df

,0,1,2,3
0,Virtual Monday: Earth Boys and Galcher Lustwerk,Nowadays,"Mon, 20 Apr 2020 /",1.0
1,Virtual Monday: Nowadays and Covid-19: What No...,Nowadays,"Mon, 20 Apr 2020 /",1.0
2,Cancelled,Bossa Nova Civic Club,"Mon, 20 Apr 2020 /",1.0
3,Virtual Monday: Nowadays and Covid-19: What No...,Nowadays,"Tue, 21 Apr 2020 /",1.0
4,Virtual Tuesday: Expert Advice with Legal Serv...,Nowadays,"Tue, 21 Apr 2020 /",1.0
5,Virtual Wednesday: Planetarium with DJ Healthy...,Nowadays,"Wed, 22 Apr 2020 /",NaN
6,Juana Molina - In The Round,Le Poisson Rouge,"Thu, 23 Apr 2020 /",NaN
7,Virtual Thursday: DJ Stingray and Loka,Nowadays,"Thu, 23 Apr 2020 /",NaN
8,rRoxymore [POSTPONED],public records,"Fri, 24 Apr 2020 /",8.0
9,Mr Bongo Records Night,Black Flamingo,"Fri, 24 Apr 2020 /",1.0


In [13]:
def scrape_events(events_page_url):
    #Your code here
    response = requests.get(events_page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    entries = event_listings.findAll('li')
    rows = []
    for entry in entries:
        #Is it a date? If so, set current date.
        date = entry.find('p', class_="eventDate date")
        event = entry.find('h1', class_="event-title")
        if event:
            details = event.text.split(' at ')
            event_name = details[0].strip()
            venue = details[1].strip()
            try:
                n_attendees = int(re.match("(\d*)", entry.find('p', class_="attending").text)[0])
            except:
                n_attendees = np.nan
            rows.append([event_name, venue, cur_date, n_attendees])
        elif date:
            cur_date = date.text
        else:
            continue
    df = pd.DataFrame(rows)
    df.head()
    df.columns = ["Event_Name", "Venue", "Event_Date", "Number_of_Attendees"]
    return df

## Write a Function to Retrieve the URL for the Next Page

In [14]:
soup.find('a', attrs={'ga-event-action':"Next "}).attrs['href']


'/events/us/newyork/week/2020-04-27'

In [15]:
def next_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    url_ext = soup.find('a', attrs={'ga-event-action':"Next "}).attrs['href']
    next_page_url = "https://www.residentadvisor.net" + url_ext
    #Your code here
    return next_page_url

## Scrape the Next 1000 Events for Your Area

Display the data sorted by the number of attendees. If there is a tie for the number attending, sort by event date.

In [16]:
dfs = []
total_rows = 0
cur_url = "https://www.residentadvisor.net/events/us/newyork"
while total_rows <= 1000:
    df = scrape_events(cur_url)
    dfs.append(df)
    total_rows += len(df)
    cur_url = next_page(cur_url)
    time.sleep(.2)
df = pd.concat(dfs)
df = df.iloc[:1000]
print(len(df))
df



1000


,Event_Name,Venue,Event_Date,Number_of_Attendees
0,Virtual Monday: Earth Boys and Galcher Lustwerk,Nowadays,"Mon, 20 Apr 2020 /",1.0
1,Virtual Monday: Nowadays and Covid-19: What No...,Nowadays,"Mon, 20 Apr 2020 /",1.0
2,Cancelled,Bossa Nova Civic Club,"Mon, 20 Apr 2020 /",1.0
3,Virtual Monday: Nowadays and Covid-19: What No...,Nowadays,"Tue, 21 Apr 2020 /",1.0
4,Virtual Tuesday: Expert Advice with Legal Serv...,Nowadays,"Tue, 21 Apr 2020 /",1.0
...,...,...,...,...
20,Virtual Saturday: Hank Jackson and Umfang b2b ...,Nowadays,"Sat, 25 Apr 2020 /",NaN
21,[POSTPONED] Teksupport x Maceo Plex present: E...,TBA - New York,"Sat, 25 Apr 2020 /",NaN
22,[CANCELED] Lost Soul presents Detroit in Effec...,Good Room,"Sat, 25 Apr 2020 /",NaN
23,[CANCELED] Elevation presents: Russian Invasio...,House Of Yes,"Sat, 25 Apr 2020 /",NaN


## Summary 

Congratulations! In this lab, you successfully developed a pipeline to scrape a website for concert event information!